<a href="https://colab.research.google.com/github/onelgo/onelgo/blob/main/calendar_backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
pip install Flask Flask-SQLAlchemy

In [20]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///calendar.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

In [21]:
class Event(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(100), nullable=False)
    description = db.Column(db.Text, nullable=True)
    date = db.Column(db.Date, nullable=False)

    def to_dict(self):
        return {
            "id": self.id,
            "title": self.title,
            "description": self.description,
            "date": self.date.strftime("%Y-%m-%d")
        }

# Initialize the database (run this once to create the database)
with app.app_context():
    db.create_all()

In [22]:
# Create a new event
@app.route('/events', methods=['POST'])
def create_event():
    data = request.get_json()
    new_event = Event(
        title=data['title'],
        description=data.get('description'),
        date=datetime.strptime(data['date'], "%Y-%m-%d")
    )
    db.session.add(new_event)
    db.session.commit()
    return jsonify({"message": "Event created!", "event": new_event.to_dict()}), 201

# Get all events
@app.route('/events', methods=['GET'])
def get_events():
    events = Event.query.all()
    return jsonify([event.to_dict() for event in events])

# Get a single event by ID
@app.route('/events/<int:event_id>', methods=['GET'])
def get_event(event_id):
    event = Event.query.get_or_404(event_id)
    return jsonify(event.to_dict())

# Update an event
@app.route('/events/<int:event_id>', methods=['PUT'])
def update_event(event_id):
    data = request.get_json()
    event = Event.query.get_or_404(event_id)
    event.title = data['title']
    event.description = data.get('description')
    event.date = datetime.strptime(data['date'], "%Y-%m-%d")
    db.session.commit()
    return jsonify({"message": "Event updated!", "event": event.to_dict()})

# Delete an event
@app.route('/events/<int:event_id>', methods=['DELETE'])
def delete_event(event_id):
    event = Event.query.get_or_404(event_id)
    db.session.delete(event)
    db.session.commit()
    return jsonify({"message": "Event deleted!"})

In [23]:
flask run

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [24]:
POST /events
{
    "title": "Birthday",
    "description": "Ali's Birthday",
    "date": "2014-12-01"
}


NameError: name 'POST' is not defined

In [ ]:
GET /events